In [1]:
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X[:3]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [3]:
y[:3]

0    0
1    0
2    0
Name: target, dtype: int64

In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from hypergbm.estimators import XGBoostEstimator
from hypergbm.pipeline import Pipeline
from hypergbm.sklearn.transformers import FeatureGenerationTransformer
from hypernets.core.ops import ModuleChoice, HyperInput
from hypernets.core.search_space import HyperSpace
from hypernets.tabular.column_selector import column_exclude_datetime


def search_space(task=None):
    space = HyperSpace()
    with space.as_default():
        input = HyperInput(name='input1')
        feature_gen = FeatureGenerationTransformer(task=task,
                                                   trans_primitives=["add_numeric", "subtract_numeric", "divide_numeric", "multiply_numeric"])  # Add feature generation to search space
        full_pipeline = Pipeline([feature_gen], name=f'feature_gen_and_preprocess', columns=column_exclude_datetime)(input)
        xgb_est = XGBoostEstimator(fit_kwargs={})
        ModuleChoice([xgb_est], name='estimator_options')(full_pipeline)
        space.set_inputs(input)
    return space

In [5]:
from hypergbm import HyperGBM
from hypernets.searchers.evolution_searcher import EvolutionSearcher

rs = EvolutionSearcher(search_space,  200, 100, optimize_direction='max')
hk = HyperGBM(rs, task='multiclass', reward_metric='accuracy', callbacks=[])
hk.search(X_train, y_train, X_eval=X_test, y_eval=y_test)

17:21:39 I hypernets.m.hyper_model.py 249 - 3 class detected, inferred as a [multiclass classification] task
17:21:39 I hypernets.c.meta_learner.py 22 - Initialize Meta Learner: dataset_id:74e7c134740a0f846f4c5e57fa5e6c93


,trial No.,Previous reward,Best trial,Best reward,Total elapsed,Max trials
0,1,0,None,None,0.002568,10


#### Current Trial:

,Trial No.,Reward,Elapsed,Space Vector
0,1,1.0,0.385699,[]


Elapsed: 00:00 | Progress: 100%|██████████
Elapsed: 00:00 | Progress: 100%|██████████
Elapsed: 00:00 | Progress:   0%|          

Pass classes=[0 1 2], y=22     0
15     0
65     1
11     0
42     0
      ..
71     1
106    2
14     0
92     1
102    2
Name: target, Length: 120, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error


Elapsed: 00:00 | Progress: 100%|██████████


#### Best Trial:

key,value
signature,d41d8cd98f00b204e9800998ecf8427e
vectors,[]


17:21:40 I hypernets.d.in_process_dispatcher.py 119 - Trial 1 done, reward: 1.0, best_trial_no:1, best_reward:1.0

17:21:40 I hypernets.d.in_process_dispatcher.py 41 - Unable to take valid sample and exceed the retry limit 1000.


In [6]:
estimator = hk.load_estimator(hk.get_best_trial().model_file)
y_pred = estimator.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

1.0